## iii-org rancher pipeline data generator example

透過服務函式建立完整rancher pipeline資料內容(教學範例)

#### 參考
* [使用-python-讀取-yaml-檔案-d3f413d7dd6](https://medium.com/bryanyang0528/%E4%BD%BF%E7%94%A8-python-%E8%AE%80%E5%8F%96-yaml-%E6%AA%94%E6%A1%88-d3f413d7dd6)
* [yaml.readthedocs](https://yaml.readthedocs.io/en/latest/example.html)

## 引入函式庫

In [1]:
import yaml
import sys
import copy

## 讀取標準範例樣板

In [2]:
with open('./stander-rancher-pipeline.yaml', 'r') as stream:
    pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)

## 顯示pipeline樣板的模樣

In [3]:
yaml.dump(pipeline_template, sys.stdout)

stages:
- name: Publish User Web image
  steps:
  - env:
      PLUGIN_INSECURE: 'true'
    publishImageConfig:
      buildContext: .
      dockerfilePath: ./Dockerfile
      pushRemote: true
      registry: 10.50.1.63:5443
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
- name: Deploy app-web
  steps:
  - applyAppConfig:
      answers:
        checkmarx.enabled: false
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
        git.url: ${CICD_GIT_URL}
        newman.enabled: false
        web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
        web.tag: latest
        web.type: uwsgi-flask
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0
- name: Deploy app-db
  steps:
  - applyAppConfig:
      answers:
        db.name: health
        db.password: bearisbear
        db.type: mysq

## 使用者僅需要前端部屬

In [4]:
def web_service_pipeline(registry_ip, ):
    with open('./stander-rancher-pipeline.yaml', 'r') as stream:
        pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
    temp_pipeline_stages = pipeline_template['stages']
    # 把最後面的階段移除
    temp_pipeline_stages.pop(2)
    return yaml.dump(pipeline_template, sys.stdout)

In [5]:
web_service_pipeline()

stages:
- name: Publish User Web image
  steps:
  - env:
      PLUGIN_INSECURE: 'true'
    publishImageConfig:
      buildContext: .
      dockerfilePath: ./Dockerfile
      pushRemote: true
      registry: 10.50.1.63:5443
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
- name: Deploy app-web
  steps:
  - applyAppConfig:
      answers:
        checkmarx.enabled: false
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
        git.url: ${CICD_GIT_URL}
        newman.enabled: false
        web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
        web.tag: latest
        web.type: uwsgi-flask
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0


## 使用者僅需要資料庫

In [6]:
def db_service_pipeline():
    with open('./stander-rancher-pipeline.yaml', 'r') as stream:
        pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
    temp_pipeline_stages = pipeline_template['stages']
    # 把最前面的兩個階段移除
    temp_pipeline_stages.pop(0)
    temp_pipeline_stages.pop(0)
    return yaml.dump(pipeline_template, sys.stdout)

In [7]:
db_service_pipeline()

stages:
- name: Deploy app-db
  steps:
  - applyAppConfig:
      answers:
        db.name: health
        db.password: bearisbear
        db.type: mysql
        db.username: bear
        git.branch: ${CICD_GIT_BRANCH}
        git.host: 10.50.1.53
        git.repoName: ${CICD_GIT_REPO_NAME}
      catalogTemplate: cattle-global-data:iii-dev-charts3-db
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0
